# Homework #6: Text Analysis

__DUE:__ Monday 2/26 11:59am

__HOW TO SUBMIT:__ __We are moving to a new submission system that is powered by git__ We are no longer going to use [WebSubmit](https://www.cs.duke.edu/csed/websubmit/). We have created a SUBMIT repository for almost every one on [coursework gitlab](https://coursework.cs.duke.edu). Make sure you have access to CS216-s2018-stu/your_firstname-your_lastname-SUBMIT; otherwise please contact zjmiao@cs.duke.edu as soon as possible. 

_If this is the first time you are submitting a hw or lab assignment:_

Fire up your VM, open a terminal, type in the following commands to initialize your SUBMIT repository:
```shell
cd ~
git clone git@coursework.cs.duke.edu:CS216-s2018-stu/<Firstname>-<Lastname>-SUBMIT.git SUBMIT
cd SUBMIT
touch README.md
git add README.md
git commit -m "add README"
git push -u origin master
```
Replace `<Firstname>` and `<Lastname>` with your name as it appears on the coursework gitlab page. If this is successful, you should see a new folder SUBMIT in your home directory. 

_How to submit an assignment:_ 

Create a new folder `hw05` directly inside your SUBMIT directory. 
```shell
cd ~/SUBMIT
mkdir hw06
```

And copy all required files (from your local `working` directory, see __WHAT TO SUBMIT__ under each problem below) into it. The sample folder structure should look like this:
```shell
/SUBMIT
/SUBMIT/hw06
/SUBMIT/hw06/hw06-q1.ipynb
/SUBMIT/hw06/hw06-q2.ipynb
...
```    

Finally, when your are ready to submit your homework, run the commands below:
```shell
cd ~/SUBMIT
git add hw06
git commit -m "updated hw06"
git push
```
You can replace "updated hw06" with any meaningful message you want.

If your submission is successful, you will see your files in your repository on gitlab website.

You can submit multiple times, but __we will only grade the files under /hw06 directory__.

# 0. Getting Started

__WHAT TO SUBMIT:__ Nothing is required for this part.

To get ready for this assignment, open up a VM shell, and type the following command:

```shell
~/CS216-s2018-READONLY/sync.sh
```

Next, type the following commands to create a working directory for this homework. Here we use `hw06` under your `working/` directory, but feel free to change it to another location.

```shell
cd ~
cp -r ~/CS216-s2018-READONLY/assignments/hw06 ~/working/
cd ~/working/hw06
```

We will be working with the same products dataset introducted in Lab #3, focusing on the Amazon products this time.  You can use your choice of programming language for this homework (e.g., SQL or python).  If you choose to use SQL, you can access the database using: 

    psql products

which should have been set up in Lab #3.  If not, you can also run `./setup.sh` (from Lab 3) to recreate the database.

If you plan to use Python for this homework, you can parse the file `amazon.ascii.csv`.

# 1. Finding Salient Words

First, we are going find salient words (appearing in the title, description or manufacturer) for each product in the amazon dataset. You will need to implement:  

__(A)__ a method for tokenizing strings to get words, and

__(B)__ a method for computing TFIDF scores for the words. 
  
You can choose to implement any method for tokenizing strings (split on spaces, punctuation, etc). We hope to use the results of this homework in Lab #6, and a good tokenizer might help you do better in the lab exercise.

_SQL HINTS:_ If you are using SQL, you will need to use string functions which can be found <a href="http://www.postgresql.org/docs/9.3/static/functions-string.html">here</a>.  You can use `regexp_split_to_table` to split a string based on a regular expression.  For example:

    SELECT id, regexp_split_to_table(title, '\s+') AS word FROM amazon;

will split the `title` string of each product in `amazon` table into substrings by whitespace, and output one row for each substring.  (You will need to tweak this query to get better tokenization.)

Also, you might want to create a view with schema `word, product_id, tfidfscore`, which can then be used in the next part.

_Python HINTS:_ If you choose to use Python, you may get the data in two ways:
(1) directly parse the data in amazon.ascii.csv using python. You can see examples in: 
* `lab05/regression.py` (which uses pandas package's read_csv command) or
* `hw05/reviews/prepare.py` (which uses the with open command)
(2) connect to the products database using the command `conn =
psycopg2.connect("dbname=products")` as we did in the `congress/prepare.py` in `hw06`, and then use SQL queries to get the data.

In Python you may use the `re.split` command to split a string based on a regular expression to get an array of substrings (described [here](https://docs.python.org/2/library/re.html)).

__WHAT TO SUBMIT:__ Create a jupyter notebook `hw06-q1.ipynb` (you can make a copy of existing any existing notebook from previous lectures). Include your code:
* If SQL: you can include the query in a Markdown cell. 
* If Python: make a code cell and include your code there. 

In [1]:
import psycopg2
import pandas as pd

/home/vcm/.local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = psycopg2.connect("dbname=products")
cursor = conn.cursor()

In [3]:
# fill in the sql query for part 1 --> creates view that part 2 will use 
q1 = """
CREATE VIEW amazon_stat (product_id, word, tfidfscore) AS
	WITH amzwords(product_id, word) AS (	
		SELECT id,
			regexp_split_to_table(title||' '||description||' '||manufacturer, E'\\s+')
			AS word
		FROM amazon),

-- IDF = number of documents word appears in

idfs(word, idf) AS (
	SELECT docw.word, COUNT(*)
	FROM (SELECT DISTINCT product_id, word FROM amzwords) AS docw
	GROUP BY word), 

-- TF = number of times word appears in a document 

tfs(product_id, word, tf) AS (
		SELECT product_id, word, COUNT(*) FROM amzwords
		GROUP BY product_id, word), 

totalnumproducts AS (SELECT COUNT(DISTINCT product_id) AS 
	numids FROM amzwords)

-- compute saliency 

SELECT tf.product_id, tf.word, tf.tf*log(totalnumproducts.numids/idf.idf) AS tfidf
	FROM tfs AS tf, idfs AS idf, totalnumproducts
	WHERE tf.word = idf.word
	ORDER BY tf.product_id, tf.tf*log(totalnumproducts.numids/idf.idf) DESC;

SELECT word, tfidfscore FROM amazon_data;
"""

pd.read_sql(q1, con=conn)

DatabaseError: Execution failed on sql '
CREATE VIEW amazon_stat (product_id, word, tfidfscore) AS
	WITH amzwords(product_id, word) AS (	
		SELECT id,
			regexp_split_to_table(title||' '||description||' '||manufacturer, E'\s+')
			AS word
		FROM amazon),

-- IDF = number of documents word appears in

idfs(word, idf) AS (
	SELECT docw.word, COUNT(*)
	FROM (SELECT DISTINCT product_id, word FROM amzwords) AS docw
	GROUP BY word), 

-- TF = number of times word appears in a document 

tfs(product_id, word, tf) AS (
		SELECT product_id, word, COUNT(*) FROM amzwords
		GROUP BY product_id, word), 

totalnumproducts AS (SELECT COUNT(DISTINCT product_id) AS 
	numids FROM amzwords)

-- compute saliency 

SELECT tf.product_id, tf.word, tf.tf*log(totalnumproducts.numids/idf.idf) AS tfidf
	FROM tfs AS tf, idfs AS idf, totalnumproducts
	WHERE tf.word = idf.word
	ORDER BY tf.product_id, tf.tf*log(totalnumproducts.numids/idf.idf) DESC;

SELECT word, tfidfscore FROM amazon_data;
': relation "amazon_sta

# 2. Finding Similar Products

Now that we have quantified the saliency of words for each product, implement an algorithm that given as input an Amazon product id, outputs the top 5 Amazon products in the database that are most similar to the input product.

For example, given product id `1931102953` as input, you might see (besides `1931102953` itself) `b000bgpqos` and `b000ivhozk` in the top 5 list.


__HINT:__ Use cosine similarity discussed in class. Here is an example output when the input id is `1931102953`. Note the similarity scores you get can be different based on how you tokenize the words. 

```
     id     |      cosine      
------------+------------------
 1931102953 | 9.63630192360657
 b000bgpqos | 3.87480547182896
 b000ivhozk | 2.28476732325212
 b000eg6d5q | 2.10493603888233
 b0002dlrru | 1.98766834498716
```

__WHAT TO SUBMIT:__ Create a jupyter notebook `hw06-q2.ipynb` (you can make a copy of existing any existing notebook from previous lectures). Include your code for computing the top 5 product ids that are most similar to an input product id.
* If SQL: you can include the query in a Markdown cell, and a sample output (for one input id) in another Markdown cell.  
* If Python: make a code cell and include your code there. Run the code for one input id. 
* _DO NOT USE THE SAME INPUT ID AS SHOWN IN THE SAMPLE OUTPUT IN THE HOMEWORK_

In [4]:
# fill in the sql query for part 2
q2 = """
-- d(D1, D2) = sum (w*y) / ((sqrt sum w^2) * (sqrt sum y^2))

SELECT y.product_id, sum(w.tfidfscore*y.tfidfscore/sqrt(botw.dim * boty.dim)) 
	AS cossimilarity
FROM amazon_stat w, amazon_stat y, 
	(SELECT product_id, sqrt(sum(tfidfscore*tfidfscore)) AS dim
		FROM amazon_stat GROUP BY product_id) AS botw, 
	(SELECT product_id, sqrt(sum(tfidfscore*tfidfscore)) AS dim
		FROM amazon_stat GROUP BY product_id) AS boty
WHERE w.product_id = '1931102953' AND w.word = y.word AND 
	w.product_id = botw.product_id AND y.product_id = boty.product_id
GROUP BY y.product_id
ORDER BY cossimilarity DESC limit 5; 

"""
pd.read_sql(q2, con=conn)

,product_id,cossimilarity
0,1931102953,9.454836
1,b000bgpqos,4.601199
2,b000ivhozk,2.768284
3,b000eg6d5q,2.585601
4,b0002dlrru,2.547438


In [5]:
# don't forget to close the connection to the database when you are done.
cursor.close()
conn.close()

# 3. Project Teams

Please submit your project teams via a text file `proj-teams.txt` in a folder named `proj-teams`. Only one person in each team needs to submit this. 